# Data pre-processing

In [1]:
from src.Utils import *

In [4]:
# Importing training dataset
TRAIN = pd.read_csv(filepath_or_buffer=Utils.FILENAMES["TRAIN"])

In [5]:
# Importing second training dataset
TRAIN_2 = pd.read_csv(filepath_or_buffer=Utils.FILENAMES["TRAIN_2"])

In [6]:
# Verify the continuity of the time series
#TRAIN.tail()
#TRAIN_2.head()

In [9]:
# Merging TRAIN and TRAIN_2 as df
df = pd.concat([TRAIN, TRAIN_2], axis = 0)

**Comments**:
- First timestamp: **2018-01-01 00:01:00**
- Last timestamp: **2022-01-24 00:00:00**

In [ ]:
# Hashmap of train datasets for each asset id
df_dict = {asset_id: None for asset_id in Utils.ASSET_IDS}

# For each asset id, perform the pre-processing
for asset_id in Utils.ASSET_IDS:
    print(f"--> Processing Asset #{asset_id} \t ({Utils.get_asset_name(asset_id)})")
    
    # Retrieving the corresponding data rows
    df_dict[asset_id] = df[df["Asset_ID"] == asset_id]
    
    # Removing the Asset_ID column (useless now)
    try:
        df_dict[asset_id].drop(["Asset_ID", "Target"], axis=1, inplace=True)
    except:
        pass
    
    # Converting the timestamp column
    df_dict[asset_id]["timestamp"] = pd.to_datetime(arg=df_dict[asset_id]["timestamp"], 
                                                     unit="s", 
                                                     errors="ignore")
    # Setting the timestamp column as index column
    df_dict[asset_id].set_index(["timestamp"], inplace=True)